In [9]:
from task2vec import Task2Vec
from models import get_model
import datasets
import task_similarity
import argparse
import torch
import torch.nn.functional as F
import torchvision.transforms as tt
from tqdm import tqdm
from synbols_utils import Synbols
import pandas as pd

In [44]:
def data_pipeline(selected_languages, selected_attrs):

    def select(data, langs, attrs):
        select = data[(data['lang'].isin(langs)) & (data['attr'].isin(attrs))]
        return select
    def merge(saved, ref):
        data = pd.DataFrame(saved)
        data['lang'] = ref['lang']
        data['attr'] = ref['attr']
        data['name'] = ref['name']
        return data

    saved = torch.load('all.pt')
    seed = 123
    ref = pd.read_csv('ref.csv', index_col = 0)
    data = merge(saved, ref)
    data = select(data=data, langs = selected_languages, attrs = selected_attrs)
    Z = [x.hessian for x in data[0].to_numpy()]
    return Z, data.reset_index(drop=True)

In [45]:
selected_languages = ['english','tamil','malayalam', 'japanese','chinese']
selected_attrs = ['natural', 'gradient', 'scaled', 'std']


In [55]:
Z, data = data_pipeline(selected_languages, selected_attrs)
Z_tensor = torch.tensor(Z)
task2vec_dim = Z_tensor.shape[1]
M = Z_tensor.shape[0] # No of Task vectors
latent_dim = 128

In [51]:
Z_tensor.shape

torch.Size([20, 13824])

In [7]:
data

,0,lang,attr,name
0,<task2vec.Embedding object at 0x7f7589d99350>,khmer,italic,khmer_italic.h5py
1,<task2vec.Embedding object at 0x7f7589d99590>,telugu,natural,telugu_natural.h5py
2,<task2vec.Embedding object at 0x7f7595ac0e10>,gujarati,italic,gujarati_italic.h5py
3,<task2vec.Embedding object at 0x7f7591ae7bd0>,chinese,gradient,chinese_gradient.h5py
4,<task2vec.Embedding object at 0x7f7594cad4d0>,tamil,italic,tamil_italic.h5py
...,...,...,...,...
130,<task2vec.Embedding object at 0x7f7589b18d50>,bangla,natural,bangla_natural.h5py
131,<task2vec.Embedding object at 0x7f7589b18e90>,english,translated,english_translated.h5py
132,<task2vec.Embedding object at 0x7f7589b18fd0>,russian,natural,russian_natural.h5py
133,<task2vec.Embedding object at 0x7f7589b1b150>,thai,italic,thai_italic.h5py
